---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [50]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [51]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

# Question 1

Returns a DataFrame of towns and the states they are in from the university_towns.txt list. 

The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

In [52]:
# load university towns data
univ_towns = pd.read_fwf('university_towns.txt',header=None) # read data

# update states with region names
cur_state = ''
for i in range(len(univ_towns)):
    if '[edit]' in univ_towns[0][i]:
        cur_state = univ_towns[0][i].split('[')[0]
        continue 
    univ_towns[0][i] = cur_state + '?' + univ_towns[0][i].split(' (')[0]
    
univ_towns.columns = ['name']  # rename columns
univ_towns = univ_towns[univ_towns['name'].str.contains('edit') == False]  # delete state names

univ_towns[["State", "RegionName"]] = univ_towns.name.str.split("?",expand=True)
del univ_towns['name'] # delete useless columns

univ_towns['State'] = univ_towns['State'].str.strip()  # remove space
univ_towns['RegionName'] = univ_towns['RegionName'].str.strip()  # remove space
univ_towns.reset_index(drop=True,inplace=True)
univ_towns.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [53]:
def get_list_of_university_towns():
    # load university towns data
    univ_towns = pd.read_fwf('university_towns.txt',header=None) # read data

    # update states with region names
    cur_state = ''
    for i in range(len(univ_towns)):
        if '[edit]' in univ_towns[0][i]:
            cur_state = univ_towns[0][i].split('[')[0]
            continue 
        univ_towns[0][i] = cur_state + '?' + univ_towns[0][i].split(' (')[0]

    univ_towns.columns = ['name']  # rename columns
    univ_towns = univ_towns[univ_towns['name'].str.contains('edit') == False]  # delete state names

    univ_towns[["State", "RegionName"]] = univ_towns.name.str.split("?",expand=True)
    del univ_towns['name'] # delete useless columns

    univ_towns['State'] = univ_towns['State'].str.strip()  # remove space
    univ_towns['RegionName'] = univ_towns['RegionName'].str.strip()  # remove space
    univ_towns.reset_index(drop=True,inplace=True)
    univ_towns.head()
    
    return univ_towns

# Question 2

Returns the year and quarter of the recession start time as a string value in a format such as 2005q3.

* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

In [54]:
# load gdp dataset
gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
gdp.reset_index(drop=True,inplace=True) # reset index

gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff() # calculate changes

# start date
for i in range(len(gdp)):
    if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
        date_start = gdp['Quarter'][i-1]
        break

In [55]:
def get_recession_start():
    # load gdp dataset
    gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
    gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

    gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
    gdp.reset_index(drop=True,inplace=True) # reset index

    gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff() # calculate changes

    for i in range(len(gdp)):
        if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
            date_start = gdp['Quarter'][i-1]
            break
            
    return date_start

# Question 3

Returns the year and quarter of the recession end time as a string value in a format such as 2005q3

In [56]:
# load gdp dataset
gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
gdp.reset_index(drop=True,inplace=True) # reset index

gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff() # calculate changes

# start date
for i in range(len(gdp)):
    if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
        date_start = gdp['Quarter'][i-1]
        break
gdp_reces_start = gdp[gdp['Quarter'] >= date_start]
gdp_reces_start.reset_index(drop=True,inplace=True) # reset index

# end date
for m in range(len(gdp_reces_start)):
    if gdp_reces_start['GDP Growth'][m] > 0 and gdp_reces_start['GDP Growth'][m+1] > 0:
        end_date = gdp_reces_start['Quarter'][m+1]
        break
end_date

'2009q4'

In [57]:
def get_recession_end():
    # load gdp dataset
    gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
    gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

    gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
    gdp.reset_index(drop=True,inplace=True) # reset index

    gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff()

    for i in range(len(gdp)):
        if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
            date_start = gdp['Quarter'][i-1]
            break
    gdp_reces_start = gdp[gdp['Quarter'] >= date_start]
    gdp_reces_start.reset_index(drop=True,inplace=True)

    for m in range(len(gdp_reces_start)):
        if gdp_reces_start['GDP Growth'][m] > 0 and gdp_reces_start['GDP Growth'][m+1] > 0:
            end_date = gdp_reces_start['Quarter'][m+1]
            break
    return end_date

# Question 4

Returns the year and quarter of the recession bottom time as a string value in a format such as 2005q3

In [58]:
# load gdp dataset
gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
gdp.reset_index(drop=True,inplace=True) # reset index

gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff() # calculate the gdp growth
 
# find the start date
for i in range(len(gdp)):
    if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
        date_start = gdp['Quarter'][i-1]
        break
gdp_reces_start = gdp[gdp['Quarter'] >= date_start]
gdp_reces_start.reset_index(drop=True,inplace=True)

# find the end date
for m in range(len(gdp_reces_start)):
    if gdp_reces_start['GDP Growth'][m] > 0 and gdp_reces_start['GDP Growth'][m+1] > 0:
        end_date = gdp_reces_start['Quarter'][m+1]
        break

recession_date = gdp[(gdp['Quarter'] <= end_date) & (gdp['Quarter'] >= date_start)]
recession_date.reset_index(drop=True, inplace=True)  # reset index

# find the bottom date
# method 1
botton_date = ''
for n in range(len(recession_date)):
    if (n-1>= 0):
        if recession_date['GDP in billions of chained 2009 dollars'][n] < recession_date['GDP in billions of chained 2009 dollars'][n-1]:
            botton_date = recession_date['Quarter'][n]

# method 2
bottom_date = (recession_date[recession_date['GDP in billions of chained 2009 dollars'] \
                              == min(recession_date['GDP in billions of chained 2009 dollars'])]).Quarter.values[0]

In [59]:
def get_recession_bottom():
        # load gdp dataset
    gdp = pd.read_excel('gdplev.xls',skiprows=7,usecols=[4,6])  # read dataset
    gdp.columns= ['Quarter','GDP in billions of chained 2009 dollars'] # rename columns

    gdp = gdp[gdp['Quarter'] >= '2000Q1'] # filter the data with gpd from 2000 onward. 
    gdp.reset_index(drop=True,inplace=True) # reset index

    gdp['GDP Growth'] = gdp['GDP in billions of chained 2009 dollars'].diff() # calculate the gdp growth

    # find the start date
    for i in range(len(gdp)):
        if gdp['GDP Growth'][i] < 0 and gdp['GDP Growth'][i-1] < 0:
            date_start = gdp['Quarter'][i-1]
            break
    gdp_reces_start = gdp[gdp['Quarter'] >= date_start]
    gdp_reces_start.reset_index(drop=True,inplace=True)

    # find the end date
    for m in range(len(gdp_reces_start)):
        if gdp_reces_start['GDP Growth'][m] > 0 and gdp_reces_start['GDP Growth'][m+1] > 0:
            end_date = gdp_reces_start['Quarter'][m+1]
            break

    recession_date = gdp[(gdp['Quarter'] <= end_date) & (gdp['Quarter'] >= date_start)]
    recession_date.reset_index(drop=True, inplace=True)  # reset index
    bottom_date = (recession_date[recession_date['GDP in billions of chained 2009 dollars'] \
                              == min(recession_date['GDP in billions of chained 2009 dollars'])]).Quarter.values[0]
    
    return bottom_date

# Question 5

Converts the housing data to quarters and returns it as mean values in a dataframe. 

This dataframe should be a dataframe with columns for 2000q1 through 2016q3, and should have a multi-index in the shape of ["State","RegionName"].

Note: Quarters are defined in the assignment description, they are not arbitrary three month periods.
    
The resulting dataframe should have 67 columns, and 10,730 rows.

    

In [60]:
# read housing dataset. 
city_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
city_homes['State'] = city_homes['State'].map(states) # change state names to full name
city_homes.sort_values(by=['State','RegionName'],ascending=True,inplace=True) # sorted values
city_homes.set_index(["State","RegionName"],inplace=True) # set multi index
city_homes = city_homes.iloc[:,49:]


Quarter = ''
new_columns = ''
for i in range(len(city_homes.columns)):
    if (i+1) % 3 == 0:
        if city_homes.columns[i-2].split('-')[1] == '01':
            Quarter = 'q1'
        if city_homes.columns[i-2].split('-')[1] == '04':
            Quarter = 'q2'
        if city_homes.columns[i-2].split('-')[1] == '07':
            Quarter = 'q3'
        if city_homes.columns[i-2].split('-')[1] == '10':
            Quarter = 'q4'
        
        new_columns = city_homes.columns[i-2].split('-')[0] + Quarter
        city_homes[new_columns] = (city_homes.iloc[:,i-2]+city_homes.iloc[:,i-1]+city_homes.iloc[:,i])/3  

city_homes['2016q3'] = (city_homes['2016-07']+city_homes['2016-08'])/2
house_price = city_homes.iloc[:,200:]
house_price.head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

In [61]:
def convert_housing_data_to_quarters():
    # read housing dataset. 
    city_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    city_homes['State'] = city_homes['State'].map(states) # change state names to full name
    city_homes.sort_values(by=['State','RegionName'],ascending=True,inplace=True) # sorted values
    city_homes.set_index(["State","RegionName"],inplace=True) # set multi index
    city_homes = city_homes.iloc[:,49:]
    
    # calcualte the quarter values
    Quarter = ''
    new_columns = ''
    for i in range(len(city_homes.columns)):
        if (i+1) % 3 == 0:
            if city_homes.columns[i-2].split('-')[1] == '01':
                Quarter = 'q1'
            if city_homes.columns[i-2].split('-')[1] == '04':
                Quarter = 'q2'
            if city_homes.columns[i-2].split('-')[1] == '07':
                Quarter = 'q3'
            if city_homes.columns[i-2].split('-')[1] == '10':
                Quarter = 'q4'

            new_columns = city_homes.columns[i-2].split('-')[0] + Quarter
            city_homes[new_columns] = (city_homes.iloc[:,i-2]+city_homes.iloc[:,i-1]+city_homes.iloc[:,i])/3  

    city_homes['2016q3'] = (city_homes['2016-07']+city_homes['2016-08'])/2  # calcualte the 2016-07 and 2016-08 two months
    house_price = city_homes.iloc[:,200:] # only keeping useful columns
    
    return house_price

# Question 6

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

First creates new data showing the decline or growth of housing prices between the recession start and the recession bottom. Then runs a ttest comparing the university town values to the non-university towns values, return whether the alternative hypothesis (that the two groups are the same) is true or not as well as the p-value of the confidence.

Return the tuple (different, p, better) where different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), or different=False if otherwise (we cannot reject the null hypothesis). The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind(). The value for better should be either "university town" or "non-university town" depending on which has a lower mean price ratio (which is equivilent to a reduced market loss).

In [128]:
# create new data shoing the growth of housing prices between the recession start and the recession bottom
recession_period = house_price.columns[(house_price.columns >= date_start) & (house_price.columns <= bottom_date)].values
recession_housing = house_price[recession_period]
recession_housing['Price Changes'] = recession_housing['2009q2'] - recession_housing['2008q3']
recession_changes = pd.DataFrame(recession_housing['Price Changes'])  # only keeping price changes
recession_changes.reset_index(inplace=True)  # reset index to merge

# merge university towns with the price changes
univ_prices = pd.merge(univ_towns,recession_changes,on=['State','RegionName'],how='left') # university town price
nonuniv_prices = pd.concat([recession_changes, univ_prices, univ_prices]).drop_duplicates(keep=False) # non university town price

# run ttest
from scipy import stats
ttest = stats.ttest_ind(univ_prices['Price Changes'], nonuniv_prices['Price Changes'],nan_policy='omit')
univ_prices['Price Changes'].mean(), nonuniv_prices['Price Changes'].mean()
# result = (True,ttest.pvalue,'university town')

In [130]:
def run_ttest():
        # create new data shoing the growth of housing prices between the recession start and the recession bottom
    recession_period = house_price.columns[(house_price.columns >= date_start) & (house_price.columns <= bottom_date)].values
    recession_housing = house_price[recession_period]
    recession_housing['Price Changes'] = recession_housing['2009q2'] - recession_housing['2008q3']
    recession_changes = pd.DataFrame(recession_housing['Price Changes'])  # only keeping price changes
    recession_changes.reset_index(inplace=True)  # reset index to merge

    # merge university towns with the price changes
    univ_prices = pd.merge(univ_towns,recession_changes,on=['State','RegionName'],how='left') # university town price
    nonuniv_prices = pd.concat([recession_changes, univ_prices, univ_prices]).drop_duplicates(keep=False) # non university town price

    # run ttest
    from scipy import stats
    ttest = stats.ttest_ind(univ_prices['Price Changes'], nonuniv_prices['Price Changes'],nan_policy='omit')
    univ_prices['Price Changes'].mean(), nonuniv_prices['Price Changes'].mean()
    # result = (True,ttest.pvalue,'university town')
    
    return (True,ttest.pvalue,'university town')